In [ ]:
!nvidia-smi

In [ ]:
import json
import time
import torch
import os
import pickle
import logging
import random

from tqdm import tqdm
from rank_bm25 import BM25Okapi
import numpy as np
from transformers import pipeline, AutoTokenizer, AutoModel

random.seed(42)

# Function to load data from a JSON file
def load_data(file_name):
    with open(file_name, 'r') as json_file:
        return json.load(json_file)

def create_user_product_matrix(data):
    user_ids = set()
    product_ids = set()
    matrix = {}
    doc_id = 0

    for user in data:
        user_id = user['id']
        for review in user['profile']:
            product_id = review['productAsin']
            user_ids.add(user_id)
            product_ids.add(product_id)

            # Set default values to "None" if text or title is missing or empty
            review_title = review.get('title', 'None') or 'None'
            review_text = review.get('text', 'None') or 'None'
            review_rating = review.get('rating', 'None')
            if isinstance(review_rating, float):
                review_rating = int(review_rating)

            matrix[(user_id, product_id)] = {
                "reviewTitle": review_title,
                "reviewText": review_text,
                "doc_id": doc_id,
                "reviewRating": review_rating
            }
            doc_id += 1  # Increment doc_id for the next review

    user_index = {user_id: idx for idx, user_id in enumerate(user_ids)}
    product_index = {product_id: idx for idx, product_id in enumerate(product_ids)}

    return matrix, user_index, product_index

In [ ]:
def retrieve_corpus_from_matrix(user_product_matrix):
    corpus = []
    for (user_id, product_id), review_data in user_product_matrix.items():
        review_text = review_data['reviewText'].strip()
        review_title = review_data['reviewTitle'].strip()
        review_rating = review_data['reviewRating']
        doc_id = review_data['doc_id']
        
        # Include review if either text or title is not empty
        if review_text or review_title:
            combined_text = f"{review_title} {review_text}".strip()
            corpus.append({
                "doc_id": doc_id,
                "user_id": user_id,
                "product_id": product_id,
                "reviewText": review_text,
                "reviewTitle": review_title,
                "reviewRating": review_rating,
                "combined": combined_text  # Combined for ranking purposes
            })
    
    return corpus
    
def load_or_create(
    file_path, 
    compute_func, 
    load_func=pickle.load, 
    save_func=pickle.dump, 
    overwrite=False, 
    compute_args=(), 
    compute_kwargs={}
):
    """
    Loads an object from a file if it exists; otherwise, computes it using the provided function,
    saves it to the file, and returns it.
    
    Parameters:
        file_path (str): Path to the file where the object is saved/loaded.
        compute_func (callable): Function to compute/build the object if not loaded.
        load_func (callable, optional): Function to load the object from file. Defaults to pickle.load.
        save_func (callable, optional): Function to save the object to file. Defaults to pickle.dump.
        overwrite (bool, optional): Whether to overwrite the existing file. Defaults to False.
        compute_args (tuple, optional): Positional arguments to pass to compute_func.
        compute_kwargs (dict, optional): Keyword arguments to pass to compute_func.
        
    Returns:
        Any: The loaded or computed object.
    """
    try:
        if os.path.exists(file_path) and not overwrite:
            logging.info(f"Loading object from {file_path}...")
            with open(file_path, 'rb') as f:
                obj = load_func(f)
            logging.info(f"Loaded object from {file_path}")
        else:
            if os.path.exists(file_path) and overwrite:
                logging.info(f"Overwriting existing file at {file_path}...")
            else:
                logging.info(f"No file found at {file_path}. Computing object...")
            
            obj = compute_func(*compute_args, **compute_kwargs)
            
            with open(file_path, 'wb') as f:
                save_func(obj, f)
            logging.info(f"Saved object to {file_path}")
        
        return obj
    except Exception as e:
        logging.error(f"Error in load_or_create for {file_path}: {e}")
        raise
        
def compute_corpus_embeddings(corpus, contriever_model, tokenizer, batch_size=32):
    corpus_embeddings = {}
    contriever_model.to(device)  # Move the model to the correct device

    for i in tqdm(range(0, len(corpus), batch_size), desc="Computing Embeddings"):
        batch = corpus[i:i+batch_size]
        texts = [doc['combined'] for doc in batch]
        doc_ids = [doc['doc_id'] for doc in batch]

        # Tokenize the batch
        encoded_input = tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)

        try:
            with torch.no_grad():
                embeddings = contriever_model(**encoded_input).pooler_output  # Shape: (batch_size, hidden_size)
        except Exception as e:
            logging.error(f"Error during embedding computation: {e}")
            continue

        embeddings = embeddings.cpu().numpy()

        for doc_id, emb in zip(doc_ids, embeddings):
            corpus_embeddings[doc_id] = emb

    return corpus_embeddings


def process_embedding(emb, device="cpu"):
    if isinstance(emb, np.ndarray):
        emb = torch.tensor(emb)
    if emb.ndim > 1:
        emb = emb.squeeze(0)
    return emb.to(device)

In [ ]:
def pgraph_rag_neighbors_ratings_only(
    user_id, 
    product_id, 
    user_product_matrix, 
    user_index, 
    product_index, 
    query, 
    corpus_embeddings,  # Precomputed embeddings
    corpus,             # Corpus should match embeddings by doc_id
    limit=None, 
    retrieval_method="con", 
    filter_field="reviewTitle"
):
    # Retrieve the product index
    product_idx = product_index.get(product_id)
    if product_idx is None:
        print(f"Product {product_id} not found in product_index.")
        return []
    
    # Get the user IDs who reviewed the product
    user_ids = [uid for (uid, pid) in user_product_matrix.keys() if pid == product_id]
    
    # Get the corresponding user IDs and their review details for the product
    neighbor_ratings = [
        {
            "user_id": uid,
            "reviewRating": user_product_matrix[(uid, product_id)]['reviewRating'],
            "reviewTitle": user_product_matrix[(uid, product_id)]['reviewTitle'], 
            "reviewText": user_product_matrix[(uid, product_id)]['reviewText'],
            "doc_id": user_product_matrix[(uid, product_id)]['doc_id']
        }
        for uid in user_ids
    ]
    
    # Filter out the current user's own review by excluding any reviews from the user_id
    filtered_ratings = [
        review for review in neighbor_ratings
        if review['user_id'] != user_id
    ]
    
    if not filtered_ratings:
        print("No neighbor ratings after filtering.")
        return []
    
    # If only one document is left, return it
    if len(filtered_ratings) == 1:
        return filtered_ratings
    
    if retrieval_method == "con":
        if corpus_embeddings is None:
            raise ValueError("Corpus embeddings must be provided for Contriever retrieval.")
        
        # Retrieve embeddings using doc_id
        filtered_embeddings = []
        for review in filtered_ratings:
            doc_id = review['doc_id']
            emb = corpus_embeddings[doc_id]
            emb = process_embedding(emb)
            filtered_embeddings.append(emb)
        
        if not filtered_embeddings:
            print("No valid embeddings for filtered documents.")
            return []
        
        # Encode the query to get the query embedding
        query_embedding = encode_for_contriever(query)
        
        # Perform Contriever retrieval using the precomputed embeddings
        top_k_indices = contriever(query_embedding, filtered_embeddings, k=limit if limit else len(filtered_embeddings))
        
        # Map back the indices to the filtered ratings
        top_k_neighbors = [filtered_ratings[i] for i in top_k_indices]
    
    else:
        # Combine title and text for each review for tokenization/embedding purposes
        combined_documents = [
            f"{review['reviewTitle']} {review['reviewText']}" for review in filtered_ratings
        ]
        # Use BM25 for tokenized retrieval
        top_k_documents = bm25_retriever(query, combined_documents, k=limit if limit else len(combined_documents))
        # Map documents back to filtered_ratings
        top_k_neighbors = []
        for doc in top_k_documents:
            index = combined_documents.index(doc)
            top_k_neighbors.append(filtered_ratings[index])
    
    return top_k_neighbors

In [ ]:
def get_user_all_ratings(
    user_id, 
    product_id,  # Add product_id parameter
    user_product_matrix, 
    query, 
    corpus_embeddings,  # Precomputed embeddings
    corpus,             # Corpus should match embeddings by index
    limit=None, 
    retrieval_method="con", 
    filter_field="reviewTitle"
):
    # Retrieve all reviews by the user
    user_ratings = [
        {
            "product_id": pid,
            "reviewRating": user_product_matrix[(uid, pid)].get('reviewRating', "None"),
            "reviewTitle": user_product_matrix[(user_id, pid)].get('reviewTitle', "None"),
            "reviewText": user_product_matrix[(user_id, pid)].get('reviewText', "None"),
            "doc_id": user_product_matrix[(user_id, pid)]['doc_id']
        }
        for (uid, pid) in user_product_matrix.keys() if uid == user_id
    ]

    if not user_ratings:
        print(f"No reviews found for user {user_id}.")
        return []

    # Identify the doc_id of the query item based on product_id
    query_doc_id = None
    for review in user_ratings:
        if review['product_id'] == product_id:
            query_doc_id = review['doc_id']
            break  # Assuming the user has only one review per product

    if query_doc_id is None:
        #print(f"No matching review found for user {user_id} and product {product_id}.")
        # Optionally, proceed without excluding the query item
        filtered_ratings = user_ratings
    else:
        # Filter out the query item using doc_id
        filtered_ratings = [
            review for review in user_ratings
            if review['doc_id'] != query_doc_id
        ]

    if not filtered_ratings:
        #print("No other reviews found for the user after filtering.")
        return []

    # Combine the title and text for tokenization/embedding
    combined_documents = [
        f"{review['reviewTitle']} {review['reviewText']}" for review in filtered_ratings
    ]

    if retrieval_method == "con":
        if corpus_embeddings is None:
            raise ValueError("Corpus embeddings must be provided for Contriever retrieval.")

        # Retrieve embeddings using doc_id
        filtered_embeddings = []
        for review in filtered_ratings:
            doc_id = review['doc_id']
            emb = corpus_embeddings[doc_id]
            emb = process_embedding(emb)
            filtered_embeddings.append(emb)

        if not filtered_embeddings:
            print("No valid embeddings for filtered documents.")
            return []

        # Encode the query to get the query embedding
        query_embedding = encode_for_contriever(query)

        # Perform Contriever-based retrieval using precomputed embeddings
        top_k_indices = contriever(query_embedding, filtered_embeddings, k=limit if limit else len(filtered_embeddings))
        
        # Map back the indices to the filtered ratings
        top_k_user_reviews = [filtered_ratings[i] for i in top_k_indices]
    else:
        # Use BM25 for tokenized retrieval
        top_k_documents = bm25_retriever(query, combined_documents, k=limit if limit else len(combined_documents))
        # Map documents back to filtered_ratings
        top_k_user_reviews = []
        for doc in top_k_documents:
            index = combined_documents.index(doc)
            top_k_user_reviews.append(filtered_ratings[index])

    return top_k_user_reviews

In [ ]:
import torch

def mean_pooling(token_embeddings, mask):
    # Mask out padded tokens and calculate mean for non-padded tokens
    token_embeddings = token_embeddings.masked_fill(~mask[..., None].bool(), 0.)
    sentence_embeddings = token_embeddings.sum(dim=1) / mask.sum(dim=1)[..., None]
    return sentence_embeddings

def encode_for_contriever(text):
    inputs = contriever_tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    
    with torch.no_grad():
        contriever_model.to(device)  # Ensure model is on the correct device
        outputs = contriever_model(**inputs)
        
        # Apply mean pooling on the token embeddings with attention mask
        embeddings = mean_pooling(outputs.last_hidden_state, inputs['attention_mask'])
    
    return embeddings.squeeze(0).to(device)  # Ensure shape is [D]


def contriever(query_embedding, document_embeddings, k=1):
    # Ensure query_embedding is a tensor on the correct device
    if not isinstance(query_embedding, torch.Tensor):
        raise ValueError(f"Expected query_embedding to be a tensor, but got {type(query_embedding)}")
    query_embedding = query_embedding.to(device)
    
    # Ensure document_embeddings are tensors on the correct device
    document_embeddings = [emb.to(device) if isinstance(emb, torch.Tensor) else torch.tensor(emb).to(device) for emb in document_embeddings]

    # Stack embeddings into tensor of shape [N, D]
    document_embeddings = torch.stack(document_embeddings)
    
    # Calculate cosine similarities between query and document embeddings
    similarities = torch.nn.functional.cosine_similarity(document_embeddings, query_embedding.unsqueeze(0), dim=1)
    similarities = similarities.cpu().numpy()
    
    # Ensure similarities is a 1D array
    similarities = similarities.squeeze()

    # Handle potential NaN values in similarities
    if np.isnan(similarities).any():
        print("Similarities contain NaN values. Replacing NaNs with zeros.")
        similarities = np.nan_to_num(similarities)

    # Get the indices of documents sorted by similarity
    top_k_indices = np.argsort(similarities)[::-1][:k]
    top_k_indices = [int(i) for i in top_k_indices]
    
    return top_k_indices  # Return the indices of the top-k most similar documents


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

def build_bm25_model(corpus, tokenize_func):
    combined_documents = [doc["combined"] for doc in corpus]
    tokenized_corpus = [tokenize_func(doc) for doc in combined_documents]
    bm25_model = BM25Okapi(tokenized_corpus)
    return bm25_model

def tokenize_for_bm25(text):
    tokens = word_tokenize(text.lower())
    return tokens

def bm25_retriever(query, documents, k=1):
    if not documents:
        print("No valid documents to retrieve.")
        return []  # Return an empty list if all documents were filtered out

    # Tokenize the filtered documents and the query using the same tokenizer
    tokenized_documents = [tokenize_for_bm25(doc) for doc in documents]
    #print(f"First tokenized document: {tokenized_documents[0] if tokenized_documents else 'No documents'}")
    
    # Further filter out any tokenized documents that are empty
    tokenized_documents = [tokens for tokens in tokenized_documents if tokens]
    
    # Check if tokenization resulted in empty documents
    if not tokenized_documents:
        print("Tokenization resulted in no valid tokens.")
        return []  # Return an empty list if tokenization fails
    
    bm25 = BM25Okapi(tokenized_documents)
    tokenized_query = tokenize_for_bm25(query)
    #print(f"Tokenized query: {tokenized_query}")
    doc_scores = bm25.get_scores(tokenized_query)
    #print(f"Document scores: {doc_scores}")
    top_k_indices = np.argsort(doc_scores)[::-1][:k]

    return [documents[i] for i in top_k_indices]

In [ ]:
def process_item(
    item, 
    user_product_matrix, 
    user_index, 
    product_index, 
    tokenizer, 
    corpus, 
    corpus_embeddings,
    bm25_model,
    limit, 
    retrieval_method="con", 
    filter_field="reviewTitle"
):
    example_user_id = item['id']
    example_product_id = item['profile'][0]['productAsin']

    
    field_mapping = {
        'reviewTitle': 'text', #swapped the query field so ranking is based of the opposite parameter and not part of fold
        'reviewText': 'title',
        #'reviewRating': 'combined'  # Special handling
    }
    query_field = field_mapping.get(filter_field)

    if filter_field == 'reviewRating':
        # Concatenate title and text to form the query
        title = item['profile'][0].get('title', '').strip()
        text = item['profile'][0].get('text', '').strip()
        query = f"{title} {text}".strip()
    else:
        # Fetch query based on the field_mapping
        query = item['profile'][0].get(query_field, '').strip()
        
    if not query:
        print(f"Warning: No '{filter_field}' found for user {item['id']}. Skipping this item.")

    # Pass product_id to exclude the review for the current product
    user_ratings = get_user_all_ratings(
        user_id=example_user_id,
        product_id=example_product_id,
        user_product_matrix=user_product_matrix,
        query=query,
        corpus_embeddings=corpus_embeddings,
        corpus=corpus,
        limit=limit,
        retrieval_method=retrieval_method,
        filter_field=filter_field
    )
    
    neighbor_ratings = pgraph_rag_neighbors_ratings_only(
        user_id=example_user_id, 
        product_id=example_product_id, 
        user_product_matrix=user_product_matrix, 
        user_index=user_index, 
        product_index=product_index, 
        query=query, 
        corpus_embeddings=corpus_embeddings, 
        corpus=corpus,
        limit=limit, 
        retrieval_method=retrieval_method, 
        filter_field=filter_field
    )
    
    # Select a random user profile from the user_product_matrix (excluding the current user)
    random_user_id = random.choice([uid for uid in user_index.keys() if uid != example_user_id])
    
    # Retrieve all reviews for the randomly selected user
    all_random_user_reviews = get_user_all_ratings(
        user_id=random_user_id,
        product_id=None,  # We want all reviews, not filtering by product
        user_product_matrix=user_product_matrix,
        query="",
        corpus_embeddings=corpus_embeddings,
        corpus=corpus,
        limit=None,  # Get all reviews, we will pick one randomly
        retrieval_method=retrieval_method,
        filter_field=filter_field
    )
    
    # Select a random review directly from all retrieved reviews
    random_review = random.choice(all_random_user_reviews) if all_random_user_reviews else None
    
    user_review_text = item['profile'][0].get('text', None)
    user_review_title = item['profile'][0].get('title', None)
    user_review_rating = item['profile'][0].get('rating', None)
    if isinstance(user_review_rating, float):
        user_review_rating = int(user_review_rating)
    
    return {
        "user_id": example_user_id,
        "product_id": example_product_id,
        "user_review_text": user_review_text,
        "user_review_title": user_review_title, 
        "user_review_rating": user_review_rating,
        "user_ratings": user_ratings,
        "neighbor_ratings": neighbor_ratings,
        "random_review": random_review
    }


def process_and_save(file_data_list, tokenizer, limit, retrieval_method="con", filter_field="reviewTitle", num_items=None):
    for data_info in tqdm(file_data_list, desc="Processing datasets", unit="dataset"):
        items = data_info['items']
        output_file = data_info['output_file']
        user_product_matrix = data_info['user_product_matrix']
        user_index = data_info['user_index']
        product_index = data_info['product_index']
        corpus = data_info['corpus']
        corpus_embeddings = data_info.get('corpus_embeddings') 
        bm25_model = data_info.get('bm25_model')
    
        print(f"Processing dataset: {output_file}")
        #print(f"Corpus embeddings is {'not None' if corpus_embeddings else 'None'}")
    
        results = []
    
        # Determine the items to process
        if num_items is not None:
            items_to_process = items[:num_items]
            print(f"Processing {len(items_to_process)} items out of {len(items)} total items.")
        else:
            items_to_process = items
            print(f"Processing all {len(items)} items.")
    
        # Inner loop to process each item in the current dataset
        print(f"\nProcessing data for {output_file}...")
        for item in tqdm(items_to_process, desc=f"Processing items in {output_file}", unit="item", leave=False):
            result = process_item(
                item=item,
                user_product_matrix=user_product_matrix,
                user_index=user_index,
                product_index=product_index,
                tokenizer=tokenizer,
                corpus=corpus,
                corpus_embeddings=corpus_embeddings,
                bm25_model=bm25_model,  
                limit=limit,
                retrieval_method=retrieval_method,
                filter_field=filter_field  # Pass the filter_field to process_item
            )
            if result is not None:
                results.append(result)
    
        # Save the results to a JSON file
        with open(output_file, 'w') as outfile:
            json.dump(results, outfile, indent=4)
    
        print(f"Ranked ratings saved to {output_file}")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

limit = 5
RETRIEVAL_METHOD = 'con'

file_path = "../data/AmazonReview/"
file_name_base = "amazon"

ranked_suffix = f"RANKING"
file_suffixes = ["test", "dev"]

# Construct the full file names with paths using the suffixes
file_names = [os.path.join(file_path, f"{file_name_base}_{suffix}.json") for suffix in file_suffixes]

# Define directories
embeddings_dir = os.path.join(file_path, "embeddings")
bm25_dir = os.path.join(file_path, "bm25_models")

# Create directories if they don't exist
os.makedirs(embeddings_dir, exist_ok=True)
os.makedirs(bm25_dir, exist_ok=True)

test_embeddings_path = os.path.join(embeddings_dir, "test_corpus_embeddings.pkl")
dev_embeddings_path = os.path.join(embeddings_dir, "dev_corpus_embeddings.pkl")
test_bm25_path = os.path.join(bm25_dir, "test_bm25_model.pkl")
dev_bm25_path = os.path.join(bm25_dir, "dev_bm25_model.pkl")

# Load the datasets
test_users = load_data(file_names[0])
dev_users = load_data(file_names[1])

test_user_product_matrix, test_user_index, test_product_index = create_user_product_matrix(test_users)
dev_user_product_matrix, dev_user_index, dev_product_index = create_user_product_matrix(dev_users)

test_corpus = retrieve_corpus_from_matrix(test_user_product_matrix)
dev_corpus = retrieve_corpus_from_matrix(dev_user_product_matrix)

In [ ]:
if RETRIEVAL_METHOD == 'con':
    # Initialize tokenizer and model
    contriever_tokenizer = AutoTokenizer.from_pretrained("facebook/contriever")
    contriever_model = AutoModel.from_pretrained("facebook/contriever").to(device)
    
    # Load or compute embeddings for Test Corpus
    test_corpus_embeddings = load_or_create(
        file_path=test_embeddings_path,
        compute_func=compute_corpus_embeddings,
        overwrite=False,
        compute_args=(test_corpus, contriever_model, contriever_tokenizer),
        compute_kwargs={'batch_size': 32}
    )
    
    # Load or compute embeddings for Dev Corpus
    dev_corpus_embeddings = load_or_create(
        file_path=dev_embeddings_path,
        compute_func=compute_corpus_embeddings,
        overwrite=False,
        compute_args=(dev_corpus, contriever_model, contriever_tokenizer),
        compute_kwargs={'batch_size': 32}
    )

elif RETRIEVAL_METHOD == 'bm25':
    # Load or build BM25 model for Test Corpus
    test_bm25_model = load_or_create(
        file_path=test_bm25_path,       # file_path
        compute_func=build_bm25_model, # compute_func
        overwrite=False,                # overwrite
        compute_args=(test_corpus, tokenize_for_bm25)  # compute_args as tuple
    )
    
    # Load or build BM25 model for Dev Corpus
    dev_bm25_model = load_or_create(
        file_path=dev_bm25_path,
        compute_func=build_bm25_model,
        overwrite=False,
        compute_args=(dev_corpus, tokenize_for_bm25)
    )
else:
    raise ValueError("Invalid retrieval method specified. Choose 'bm25' or 'con'.")

In [ ]:
# Prepare the file_data_list with BM25 or Contriever models and corpora
file_data_list = [
    {
        'items': test_users,
        'output_file_template': os.path.join(
            file_path,
            f"{ranked_suffix}-{file_name_base}_test_{{filter_field}}_{RETRIEVAL_METHOD}.json"
        ),
        'user_product_matrix': test_user_product_matrix,
        'user_index': test_user_index,
        'product_index': test_product_index,
        'corpus': test_corpus,
        'corpus_embeddings': test_corpus_embeddings if RETRIEVAL_METHOD == 'con' else None,
        'bm25_model': test_bm25_model if RETRIEVAL_METHOD == 'bm25' else None
    },
    {
        'items': dev_users,
        'output_file_template': os.path.join(
            file_path,
            f"{ranked_suffix}-{file_name_base}_dev_{{filter_field}}_{RETRIEVAL_METHOD}.json"
        ),
        'user_product_matrix': dev_user_product_matrix,
        'user_index': dev_user_index,
        'product_index': dev_product_index,
        'corpus': dev_corpus,
        'corpus_embeddings': dev_corpus_embeddings if RETRIEVAL_METHOD == 'con' else None,
        'bm25_model': dev_bm25_model if RETRIEVAL_METHOD == 'bm25' else None
    }
]

In [ ]:
# Define the filter_fields you want to process
#filter_fields = ['reviewTitle', 'reviewText', 'reviewRating']
filter_fields = ['reviewTitle']
for filter_field in filter_fields:
    print(f"\nProcessing with filter_field: {filter_field}")

    # Update the output_file for each dataset
    for data_info in file_data_list:
        data_info['output_file'] = data_info['output_file_template'].format(filter_field=filter_field)

    # Process and save the datasets for this filter_field
    process_and_save(
        file_data_list,
        tokenizer=None,
        limit=limit,
        retrieval_method=RETRIEVAL_METHOD,
        filter_field=filter_field,
        #num_items=2,
    )

In [ ]:
from IPython.display import display
from ipywidgets import Button

def shutdown_kernel():
    from IPython.display import display
    display("Shutting down kernel...")
    get_ipython().kernel.do_shutdown(True)

shutdown_kernel()
